In [ ]:
import pandas as pd #veri setini yüklemek
import re #veriyi temizlemek için
import nltk #doğal dil işleme araçları(stopwords, kök bulma)
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from google.colab import files

#apı ayarları
# Kaggle JSON'u Yükleme
files.upload()

# Kaggle JSON'u Doğru Konuma Taşıma
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Kaggle Veri Setini İndirme
!kaggle datasets download -d kazanova/sentiment140
!unzip sentiment140.zip

#  Stopwords ve Stemmer
nltk.download('stopwords') #ingilizce durak kelimeleri indiriyor
stop_words = set(stopwords.words('english'))
port_stem = PorterStemmer() #kelime kökleri

# Veri Setini Yükleme ve Hazırlama
def load_and_prepare_data(file_path):
    column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
    twitter_data = pd.read_csv(
        file_path,
        names=column_names,
        encoding='ISO-8859-1', # verileri okurken latin alfabesine göre kodlamalı
        on_bad_lines='skip' #hatalı satır varsa atla
    )

    # Metin Verisinin Hazırlanması
    twitter_data['target'] = twitter_data['target'].map(lambda x: 1 if x == 4 else 0)  # olumlu(4) 1 olarak olumsuz 0 olarak kodlanır
    twitter_data['text'] = twitter_data['text'].fillna("").apply(preprocess_text) # eksik değerleri(fillna) boş yap ve preprocess fonks. her satıra apply et

    X = twitter_data['text'].values #bağımsız değişken
    Y = twitter_data['target'].values

    return train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42) # teste ayırırken veri dağılımını koruması için

# Metni Ön İşleme Fonksiyonu
def preprocess_text(content):
    content = re.sub('[^a-zA-Z]', ' ', content)  # Harfler dışındakileri boşlukla değiştirir
    content = content.lower()  # Küçük harfe dönüştür
    words = content.split()  # Kelimelere ayır
    stemmed_words = [port_stem.stem(word) for word in words if word not in stop_words]
    return ' '.join(stemmed_words) #geri birleştirdim

# Modeli Eğitme ve Test Etme
def train_and_evaluate(file_path):
    X_train, X_test, Y_train, Y_test = load_and_prepare_data(file_path)

    # TF-IDF
    vectorizer = TfidfVectorizer(max_features=5000) #en sık geçen 5k kelime dikkate alır
    X_train = vectorizer.fit_transform(X_train) # fit:kelime frekansları belirler transform: vektör formatına çevirir
    X_test = vectorizer.transform(X_test) # x_trainde öğrenilen frekanslar ile test seti dönüştürülüyor. yeni kelime öğrenilmiyor sadece test seti işlenir

    # SVM Modeli
    model = LinearSVC()
    model.fit(X_train, Y_train) #model eğitilir

    # Tahmin ve Doğruluk
    Y_pred = model.predict(X_test)
    print(f"Accuracy: {accuracy_score(Y_test, Y_pred):.2f}")
    print("\nClassification Report:\n", classification_report(Y_test, Y_pred))

    return model, vectorizer

# Ana Fonksiyon
def main():
    file_path = "training.1600000.processed.noemoticon.csv"
    print("Model eğitiliyor, lütfen bekleyin...")
    model, vectorizer = train_and_evaluate(file_path)

    print("Model eğitimi tamamlandı!")
    while True:
        text = input("Bir metin girin ('çıkış' yazıp çıkabilirsiniz): ")
        if text.lower() == 'çıkış':
            print("Programdan çıkılıyor...")
            break

        processed_text = preprocess_text(text) #kullanıcının girdisi işlenir
        text_vector = vectorizer.transform([processed_text])
        prediction = model.predict(text_vector)[0]
        sentiment = "Olumlu" if prediction == 1 else "Olumsuz"
        print(f"Tahmin edilen duygu: {sentiment}")

if __name__ == '__main__':
    main()


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 80% 65.0M/80.9M [00:00<00:00, 201MB/s]
100% 80.9M/80.9M [00:00<00:00, 189MB/s]
Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  
Model eğitiliyor, lütfen bekleyin...


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Accuracy: 0.77

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.74      0.76    160000
           1       0.75      0.80      0.77    160000

    accuracy                           0.77    320000
   macro avg       0.77      0.77      0.77    320000
weighted avg       0.77      0.77      0.77    320000

Model eğitimi tamamlandı!
Bir metin girin ('çıkış' yazıp çıkabilirsiniz): ı love being woman
Tahmin edilen duygu: Olumlu
Bir metin girin ('çıkış' yazıp çıkabilirsiniz): I hate people
Tahmin edilen duygu: Olumsuz
Bir metin girin ('çıkış' yazıp çıkabilirsiniz): Glad to meet you
Tahmin edilen duygu: Olumlu
Bir metin girin ('çıkış' yazıp çıkabilirsiniz): I think that humanity is corrupted
Tahmin edilen duygu: Olumsuz
Bir metin girin ('çıkış' yazıp çıkabilirsiniz): çıkış
Programdan çıkılıyor...


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from google.colab import files

# Kaggle JSON'u Yükleme
files.upload()

# Kaggle JSON'u Doğru Konuma Taşıma
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Kaggle Veri Setini İndirme
!kaggle datasets download -d kazanova/sentiment140
!unzip sentiment140.zip

# NLTK Stopwords ve Stemmer
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
port_stem = PorterStemmer()

# Veri Setini Yükleme ve Hazırlama
def load_and_prepare_data(file_path):
    # Veri Seti Sütunları
    column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
    twitter_data = pd.read_csv(
        file_path,
        names=column_names,
        encoding='ISO-8859-1',
        on_bad_lines='skip'
    )

    # Hedef Sütunun ve Metin Verisinin Hazırlanması
    twitter_data['target'] = twitter_data['target'].map(lambda x: 1 if x == 4 else 0)  # 4: Olumlu, 0: Olumsuz
    twitter_data['text'] = twitter_data['text'].fillna("").apply(preprocess_text)

    X = twitter_data['text'].values
    Y = twitter_data['target'].values

    return train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

# Metni Ön İşleme Fonksiyonu
def preprocess_text(content):
    content = re.sub('[^a-zA-Z]', ' ', content)  # Sadece harfler
    content = content.lower()  # Küçük harfe dönüştür
    words = content.split()  # Kelimelere ayır
    stemmed_words = [port_stem.stem(word) for word in words if word not in stop_words]
    return ' '.join(stemmed_words)

# Modeli Eğitme ve Test Etme
def train_and_evaluate(file_path):
    X_train, X_test, Y_train, Y_test = load_and_prepare_data(file_path)

    # TF-IDF Vektörizasyon
    vectorizer = TfidfVectorizer(max_features=5000)
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Naive Bayes Modeli
    model = MultinomialNB()
    model.fit(X_train, Y_train)

    # Tahmin ve Doğruluk
    Y_pred = model.predict(X_test)
    print(f"Accuracy: {accuracy_score(Y_test, Y_pred):.2f}")
    print("\nClassification Report:\n", classification_report(Y_test, Y_pred))

    return model, vectorizer

# Ana Fonksiyon
def main():
    file_path = "training.1600000.processed.noemoticon.csv"
    print("Model eğitiliyor, lütfen bekleyin...")
    model, vectorizer = train_and_evaluate(file_path)

    print("Model eğitimi tamamlandı!")
    while True:
        text = input("Bir metin girin ('çıkış' yazıp çıkabilirsiniz): ")
        if text.lower() == 'çıkış':
            print("Programdan çıkılıyor...")
            break

        processed_text = preprocess_text(text)
        text_vector = vectorizer.transform([processed_text])
        prediction = model.predict(text_vector)[0]
        sentiment = "Olumlu" if prediction == 1 else "Olumsuz"
        print(f"Tahmin edilen duygu: {sentiment}")

if __name__ == '__main__':
    main()

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 82% 66.0M/80.9M [00:00<00:00, 240MB/s]
100% 80.9M/80.9M [00:00<00:00, 236MB/s]
Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  
Model eğitiliyor, lütfen bekleyin...


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Accuracy: 0.75

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.75      0.75    160000
           1       0.75      0.75      0.75    160000

    accuracy                           0.75    320000
   macro avg       0.75      0.75      0.75    320000
weighted avg       0.75      0.75      0.75    320000

Model eğitimi tamamlandı!
Bir metin girin ('çıkış' yazıp çıkabilirsiniz): ı love being student
Tahmin edilen duygu: Olumlu
Bir metin girin ('çıkış' yazıp çıkabilirsiniz): ı dont like this topic
Tahmin edilen duygu: Olumsuz
Bir metin girin ('çıkış' yazıp çıkabilirsiniz): I think that humanity is corrupted
Tahmin edilen duygu: Olumsuz
Bir metin girin ('çıkış' yazıp çıkabilirsiniz): çıkış
Programdan çıkılıyor...
